In [ ]:
!pip install -q condacolab
import condacolab
condacolab.install()

In [1]:
!git config --global user.name "Alex McKenzie" 
!git config --global user.email "amckenzie9@gatech.edu"
!git clone https://github.com/Arrrlex/amptorch.git && cd amptorch && git checkout fix-conda-env
!git clone https://github.com/medford-group/bdqm-vip.git
!mamba env update -n base -f amptorch/env_cpu.yml
!pip install ./amptorch

fatal: destination path 'amptorch' already exists and is not an empty directory.
Cloning into 'bdqm-vip'...
remote: Enumerating objects: 694, done.
remote: Counting objects: 100% (280/280), done.
remote: Compressing objects: 100% (227/227), done.
remote: Total 694 (delta 157), reused 117 (delta 49), pack-reused 414
Receiving objects: 100% (694/694), 143.31 MiB | 4.86 MiB/s, done.
Resolving deltas: 100% (388/388), done.


In [7]:
import torch
import ase.io
from amptorch.trainer import AtomsTrainer

from pathlib import Path

bdqm_path = Path("bdqm-vip")
amptorch_path = Path("amptorch")

train_data = ase.io.read(bdqm_path / "data/amptorch_data/oc20_3k_train.traj")

elements = list(set(atom.symbol for atom in train_data))

def get_path_to_gaussian(element):
    gaussians_path = amptorch_path / "examples/GMP/valence_gaussians"
    return next(p for p in gaussians_path.iterdir() if p.name.startswith(element + "_"))

atom_gaussians = {element: get_path_to_gaussian(element) for element in elements}

sigmas = [0.2, 0.69, 1.1, 1.66, 2.66]

MCSHs = {
    "MCSHs": {
        "0": {"groups": [1], "sigmas": sigmas},
        "1": {"groups": [1], "sigmas": sigmas},
        "2": {"groups": [1, 2], "sigmas": sigmas},
    },
    "atom_gaussians": atom_gaussians,
    "cutoff": 8,
}

train_data_path = str(bdqm_path / "data/amptorch_data/oc20_3k_train.traj")


model_config = {
    "name":"singlenn",
    "get_forces": True,
    "num_layers": 3,
    "num_nodes": 10,
    "batchnorm": True,
}

optim_config = {
    "force_coefficient": 0.01,
    "lr": 1e-3,
    "batch_size": 16,
    "epochs": 10,
    "loss": "mse",
    "metric": "mae",
}

dataset_config = {
    "raw_data": train_data_path,
    "fp_scheme": "gmp",
    "fp_params": MCSHs,
    "elements": elements,
    "save_fps": True,
    "scaling": {"type": "normalize", "range": (0, 1)},
    "val_split": 0.1,
}

cmd_config = {
    "debug": False,
    "run_dir": "./",
    "seed": 1,
    "identifier": "test",
    "verbose": True,
    "logger": False,
}

config = {
    "model": model_config,
    "optim": optim_config,
    "dataset": dataset_config,
    "cmd": cmd_config,
}

torch.set_num_threads(1)
trainer = AtomsTrainer(config)
trainer.train()

Results saved to ./checkpoints/2022-02-08-15-14-05-test


Scaling Target data: 100%|██████████| 3000/3000 [00:00<00:00, 41523.93 scalings/s]


Loading dataset: 3000 images
Loading model: 501 parameters
Loading skorch trainer
  epoch    train_energy_mae    train_forces_mae    train_loss    val_energy_mae    val_forces_mae    valid_loss    cp      dur
-------  ------------------  ------------------  ------------  ----------------  ----------------  ------------  ----  -------
      1           1217.9261            112.0899       37.5123         1512.9265           86.5026       29.8566     +  15.1829


<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


      2            626.4187             85.2256        7.9544          479.3150           71.5122        4.6715     +  15.2036
      3            540.5634             66.5865        5.8387          436.3370           57.6336        3.4785     +  15.1625
      4            467.2123             58.9983        4.0027          316.2756           58.0438        2.5031     +  15.4901
      5            402.9479             53.5226        3.0752          528.6107           58.6039        3.6555        15.4318
      6            384.3636             48.1549        2.6971          270.6585           42.5644        1.6739     +  15.4567
      7            328.0601             35.9508        2.0188          265.5656           47.5287        1.5250     +  15.5408
      8            292.3840             32.9062        1.5602          374.6311           36.3406        2.0441        15.6800
      9            272.9246             27.6154        1.3595          468.9204           38.4870        2.5853